In [1]:
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
files = [
    "files/apostila.pdf",
    "files/LLM.pdf"
]
pages = []
for file in files:
    loader = PyMuPDFLoader(file)
    pages.extend(loader.load())


chunk_size = 500
chunk_overlap = 50

recursive_split = RecursiveCharacterTextSplitter(
    chunk_overlap=chunk_overlap,
    chunk_size=chunk_size,
    separators=["\n\n","\n"," ", "", "."]
)
documents = recursive_split.split_documents(pages)
len(documents)

130

In [3]:
for i, doc in enumerate(documents):
    doc.metadata["source"] = doc.metadata["source"].replace("files/","")
    doc.metadata["doc_id"] = i

In [4]:
documents[2].metadata

{'producer': 'Microsoft® Word 2013',
 'creator': 'Microsoft® Word 2013',
 'creationdate': '2016-05-04T10:06:39-03:00',
 'source': 'apostila.pdf',
 'file_path': 'files/apostila.pdf',
 'total_pages': 28,
 'format': 'PDF 1.5',
 'title': '',
 'author': 'lucas',
 'subject': '',
 'keywords': '',
 'moddate': '2016-05-04T10:06:39-03:00',
 'trapped': '',
 'modDate': "D:20160504100639-03'00'",
 'creationDate': "D:20160504100639-03'00'",
 'page': 1,
 'doc_id': 2}

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embeddings_model = OpenAIEmbeddings()

In [6]:
directory = "files/chroma_retrival_db" 

vectordb= Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory= directory
)

## Semantic Search

In [7]:
question = "O que é um LLM?"

docs = vectordb.similarity_search(question, k=3)

for doc in docs:
    print(doc.page_content)
    print(f'=={doc.metadata}\n')

código de exemplo que podem ajudar você a começar a trabalhar com LLMs 
imediatamente.
=={'format': 'PDF 1.4', 'creator': '', 'title': '', 'keywords': '', 'moddate': "D:20231017132635Z00'00'", 'page': 7, 'author': '', 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'file_path': 'files/LLM.pdf', 'subject': '', 'doc_id': 127, 'source': 'LLM.pdf', 'modDate': "D:20231017132635Z00'00'", 'trapped': '', 'creationdate': "D:20231017132635Z00'00'", 'creationDate': "D:20231017132635Z00'00'", 'total_pages': 9}

2 
 
 
 
 
 
 
 
 
 
PARTE  1 
Introdução 
 
 
Definição de LLM (tradução livre: grandes modelos de linguagem) 
LLMs são sistemas de IA desenvolvidos para processar e analisar enormes 
quantidades de dados de linguagem natural e, em seguida, usar essas 
informações para gerar respostas às solicitações dos usuários. Esses sistemas são 
treinados em grandes conjuntos de dados usando algoritmos avançados de 
machine learning para aprender os padrões e as estruturas da linguag

### Max MArginal Relevance

In [8]:
question = "O que é um LLM?"

docs = vectordb.max_marginal_relevance_search(question, k=3, fetch_k=10)

for doc in docs:
    print(doc.page_content)
    print(f'=={doc.metadata}\n')

código de exemplo que podem ajudar você a começar a trabalhar com LLMs 
imediatamente.
=={'page': 7, 'format': 'PDF 1.4', 'trapped': '', 'moddate': "D:20231017132635Z00'00'", 'creator': '', 'creationdate': "D:20231017132635Z00'00'", 'source': 'LLM.pdf', 'keywords': '', 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'file_path': 'files/LLM.pdf', 'creationDate': "D:20231017132635Z00'00'", 'subject': '', 'total_pages': 9, 'modDate': "D:20231017132635Z00'00'", 'doc_id': 127, 'author': '', 'title': ''}

2 
 
 
 
 
 
 
 
 
 
PARTE  1 
Introdução 
 
 
Definição de LLM (tradução livre: grandes modelos de linguagem) 
LLMs são sistemas de IA desenvolvidos para processar e analisar enormes 
quantidades de dados de linguagem natural e, em seguida, usar essas 
informações para gerar respostas às solicitações dos usuários. Esses sistemas são 
treinados em grandes conjuntos de dados usando algoritmos avançados de 
machine learning para aprender os padrões e as estruturas da linguag

### Filter

In [11]:
question = "o que a apostila de LLM fala sobre OpenAi e o ChatGPT?"

docs = vectordb.similarity_search(question, 
                                  k=3,
                                  filter={"$and":[{"source":"LLM.pdf"}, 
                                          {"page":{"$in":[3,4,5,6]}}]})

for doc in docs:
    print(doc.page_content)
    print(f'=={doc.metadata}\n')

Serviços proprietários 
Como o primeiro serviço amplamente disponível alimentado por LLM, o 
ChatGPT da OpenAI foi o catalisador explosivo que trouxe os LLMs para o 
mainstream. O ChatGPT fornece uma interface de usuário (ou API) em que os 
usuários podem enviar prompts para muitos 
modelos (GPT-3.5, GPT-4 e outros) e geralmente obter uma resposta rápida. Eles 
estão entre os modelos de maior desempenho, treinados em conjuntos de dados
=={'source': 'LLM.pdf', 'title': '', 'author': '', 'creator': '', 'file_path': 'files/LLM.pdf', 'doc_id': 115, 'modDate': "D:20231017132635Z00'00'", 'moddate': "D:20231017132635Z00'00'", 'total_pages': 9, 'page': 5, 'creationDate': "D:20231017132635Z00'00'", 'trapped': '', 'creationdate': "D:20231017132635Z00'00'", 'format': 'PDF 1.4', 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'keywords': '', 'subject': ''}

AVANÇOS EM TÉCNICAS  
Nos últimos anos, houve avanços significativos nas técnicas usadas para treinar 
esses modelos, result

### LLM Aided Retrieval

In [12]:
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

In [13]:
metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Pode ser "apostila.pdf" ou "LLM.pdf".',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto foi extraído. Número da página.',
        type='integer'
    ),
]

In [14]:
document_description = "Apostilas de informações"
llm = OpenAI()

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_description,
    metadata_info,
    verbose=True
)

In [15]:
pergunta = 'O que a apostila de LLM fala sobre a OpenAI, ChatGPT e Hugging Face?'

docs = retriever.get_relevant_documents(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

/tmp/ipykernel_489108/2978584755.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(pergunta)


Serviços proprietários 
Como o primeiro serviço amplamente disponível alimentado por LLM, o 
ChatGPT da OpenAI foi o catalisador explosivo que trouxe os LLMs para o 
mainstream. O ChatGPT fornece uma interface de usuário (ou API) em que os 
usuários podem enviar prompts para muitos 
modelos (GPT-3.5, GPT-4 e outros) e geralmente obter uma resposta rápida. Eles 
estão entre os modelos de maior desempenho, treinados em conjuntos de dados
==========={'format': 'PDF 1.4', 'total_pages': 9, 'page': 5, 'modDate': "D:20231017132635Z00'00'", 'creationdate': "D:20231017132635Z00'00'", 'source': 'LLM.pdf', 'creationDate': "D:20231017132635Z00'00'", 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'keywords': '', 'title': '', 'subject': '', 'author': '', 'file_path': 'files/LLM.pdf', 'creator': '', 'moddate': "D:20231017132635Z00'00'", 'trapped': '', 'doc_id': 115}


aberto, onde houve um crescimento igualmente explosivo nos últimos anos. 
Comunidades como a Hugging Face reúnem c